# Architecture and dimensions
We use a compact encoder–GCN–decoder pipeline:

Inputs per node i
Club (categorical): embedded as a 4‑dim vector via an embedding table of size 2×4 → 8 trainable parameters.
Scalars: degree (normalized), betweenness, closeness → 3 dims.
Concatenation → x0 ∈ R^{7} per node.
Linear projection: Linear(7 → 10)
Weights: 10×7 = 70; Bias: 10 → 80 trainable parameters.
GCN layers on the (undirected) graph using PyG’s GCNConv
GCNConv(10 → 10): Weights 10×10 = 100; Bias 10 → 110 params.
GCNConv(10 → 10): Weights 10×10 = 100; Bias 10 → 110 params.
Final embedding z is the concatenation of the outputs after the first and second GCN layers → z ∈ R^{20} per node.
Decoder: dot‑product for link prediction, σ(z_i^T z_j) → probability of edge (i, j).
Parameter total (trainable): 8 (embed) + 80 (linear) + 110 (GCN1) + 110 (GCN2) = 308.

Shapes through the forward pass for N nodes:

club_emb(club_idx): [N, 4]
scalars: [N, 3]
concat: [N, 7]
after Linear(+ReLU): [N, 10]
after GCNConv1(+ReLU): [N, 10]
after GCNConv2: [N, 10]
final z = concat([after GCNConv1, after GCNConv2]): [N, 20]
Optional verification of parameter counts and shapes:

In [ ]:
try:
  import torch, torch.nn.functional as F
  from torch import nn
  from torch_geometric.nn import GCNConv
  import networkx as nx

  class KarateGCN(nn.Module):
    def __init__(self, num_clubs=2, club_emb_dim=4, scalar_dim=3, hidden_dim=10):
      super().__init__()
      self.club_emb = nn.Embedding(num_clubs, club_emb_dim)
      self.lin = nn.Linear(club_emb_dim + scalar_dim, hidden_dim)
      self.conv1 = GCNConv(hidden_dim, hidden_dim, add_self_loops=True)
      self.conv2 = GCNConv(hidden_dim, hidden_dim, add_self_loops=True)
    def forward(self, club_idx, scalars, edge_index):
      x = torch.cat([self.club_emb(club_idx), scalars], dim=1)
      x = F.relu(self.lin(x))
      x1 = self.conv1(x, edge_index)
      y = F.relu(x1)
      x2 = self.conv2(y, edge_index)
      z = torch.cat([x1, x2], dim=1)
      return x, x1, x2, z

  # Build a tiny instance on Karate Club for shape checks
  G = nx.karate_club_graph(); N = G.number_of_nodes()
  club_idx = torch.tensor([0 if G.nodes[i]['club'] == 'Mr. Hi' else 1 for i in range(N)], dtype=torch.long)
  import torch
  deg_vals = torch.tensor([G.degree(i) for i in range(N)], dtype=torch.float32); deg_vals = deg_vals/deg_vals.max()
  import networkx as nx
  betw_vals = torch.tensor(list(nx.betweenness_centrality(G).values()), dtype=torch.float32)
  close_vals = torch.tensor(list(nx.closeness_centrality(G).values()), dtype=torch.float32)
  scalars = torch.stack([deg_vals, betw_vals, close_vals], dim=1)
  import torch_geometric
  from torch_geometric.utils import to_undirected
  edges = torch.tensor(list(G.edges()), dtype=torch.long).t().contiguous(); edge_index = to_undirected(edges, num_nodes=N)

  model = KarateGCN()
  x_lin, x_gcn1, x_gcn2, z = model(club_idx, scalars, edge_index)
  total_params = sum(p.numel() for p in model.parameters())
  print("Shapes:")
  print("  after Linear:", tuple(x_lin.shape))
  print("  after GCNConv1:", tuple(x_gcn1.shape))
  print("  after GCNConv2:", tuple(x_gcn2.shape))
  print("  final z (concat):", tuple(z.shape))
  print("Total trainable parameters:", total_params)
except Exception as e:
  print("(Skipping verification — torch_geometric not available in this kernel.)")